In [1]:
from pyoculus.problems import CylindricalBfield, AnalyticCylindricalBfield
from pyoculus.solvers import PoincarePlot, FixedPoint, Manifold
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pickle

In [3]:
# separatrix = {"type": "circular-current-loop", "amplitude": -4.2, "R": 3, "Z": -2.2}
separatrix = {"type": "circular-current-loop", "amplitude": -10, "R": 6, "Z": -5.5}
# separatrix = {"type": "circular-current-loop", "amplitude": -4, "R": 3, "Z": -2.2}
maxwellboltzmann = {"m": 7, "n": -1, "d": np.sqrt(2), "type": "maxwell-boltzmann", "amplitude": 0.1, "A": 1, "B": 2}
# gaussian10 = {"m": 1, "n": 0, "d": 1, "type": "gaussian", "amplitude": 0.1}

ps = AnalyticCylindricalBfield.without_axis(6, 0, 0.91, 0.6, perturbations_args = [separatrix], Rbegin = 2, Rend = 8, niter = 800, guess=[6.4,-0.7],  tol = 1e-9)
# ps.add_perturbation(maxwellboltzmann)
# ps = AnalyticCylindricalBfield(3, 0, 0.9, 0.7, perturbations_args = [separatrix])

0 - dr : 0.02290324895580045
1 - RZ : [ 6.41398572 -0.69363305]
1 - dr : 0.0001521514183698729
2 - RZ : [ 6.41409781 -0.69367862]
2 - dr : 1.3221698258761472e-08
3 - RZ : [ 6.41409781 -0.69367863]
3 - dr : 3.568292035180542e-15


### Finding fixed points

In [3]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-7

pparams = dict()
pparams["nrestart"] = 0
pparams['niter'] = 600

fp_perturbed = FixedPoint(ps, pparams, integrator_params=iparams)

# fp_perturbed.compute(guess=[fp.x[0], fp.z[0]], pp=0, qq=1, sbegin=0.1, send=6, tol = 1e-10)
# fp_perturbed.compute(guess=[3.117263523069049, -1.6173346133145015], pp=0, qq=1, sbegin=0.1, send=6, tol = 1e-10)
# fp_perturbed.compute(guess=[3.1072023810385443, -1.655410284892828], pp=0, qq=1, sbegin=0.1, send=6, tol = 4e-12)
# fp_perturbed.compute(guess=[3.117264916246293, -1.617334822348791], pp=0, qq=1, sbegin=0.1, send=6, tol = 1e-10)
# fp_perturbed.compute(guess=[4.624454, 0.], pp=0, qq=1, sbegin=0.1, send=6, tol = 1e-10)
# fp_perturbed.compute(guess=[4.43582958 -1.22440153], pp=0, qq=1, sbegin=0.1, send=6, tol = 1e-10)
fp_perturbed.compute(guess=[6.2, -4.45], pp=0, qq=1, sbegin=1, send=8, tol = 1e-10)

0 - [DeltaR, DeltaZ] : [0.31173615 0.2138405 ] - dtheta : 0.08449022978024034
0 - [StepR, StepZ]: [-0.00118302 -0.04520414]
1 - RZ : [ 6.19881698 -4.49520414] - rhotheta : [ 3.80761632 -1.62736602]
1 - [DeltaR, DeltaZ] : [-0.11753869 -0.11778612] - dtheta : 0.02814491773408445
1 - [StepR, StepZ]: [0.00407528 0.0023742 ]
2 - RZ : [ 6.20289227 -4.49282994] - rhotheta : [ 3.80501754 -1.62633198]
2 - [DeltaR, DeltaZ] : [-0.00943189 -0.01458011] - dtheta : 0.0022533535789532877
2 - [StepR, StepZ]: [ 0.00281002 -0.00375591]
3 - RZ : [ 6.20570228 -4.49658586] - rhotheta : [ 3.80861288 -1.62554058]
3 - [DeltaR, DeltaZ] : [7.67243047e-05 2.13139128e-05] - dtheta : 1.980868804474234e-05
3 - [StepR, StepZ]: [ 1.64771197e-05 -3.42847330e-05]
4 - RZ : [ 6.20571876 -4.49662014] - rhotheta : [ 3.80864621 -1.62553576]
4 - [DeltaR, DeltaZ] : [-1.68349779e-10 -9.47335543e-10] - dtheta : 3.407696347323963e-11
4 - [StepR, StepZ]: [ 3.30419248e-10 -5.30092414e-10]
5 - RZ : [ 6.20571876 -4.49662014] - rhoth

In [4]:
results = [list(p) for p in zip(fp_perturbed.x, fp_perturbed.y, fp_perturbed.z)]

In [5]:
results

[[6.205718761264171, 0.0, -4.496620141391966],
 [6.2057187612640625, 0.0, -4.496620141392062]]

### Poincare

In [ ]:
# set up the integrator for the Poincare
iparams = dict()
iparams["rtol"] = 1e-10

# set up the Poincare plot
pparams = dict()
pparams["nPtrj"] = 8
pparams["nPpts"] = 150
pparams["zeta"] = 0

# # Set RZs for the normal (R-only) computation
# pparams["Rbegin"] = 6.3
# pparams["Rend"] = 9.1

# Set RZs for the tweaked (R-Z) computation
nfieldlines = pparams["nPtrj"] + 1

# Directly setting the RZs
# Rs = np.linspace(6, 3.15, nfieldlines)
# Zs = np.linspace(-0.43, -2.5, nfieldlines)
# RZs = np.array([[r, z] for r, z in zip(Rs, Zs)])

# Two interval computation opoint to xpoint then xpoint to coilpoint
n1, n2 = int(np.ceil(nfieldlines / 2)), int(np.floor(nfieldlines / 2))
xpoint = np.array([results[0][0], results[0][2]])
opoint = np.array([ps._R0, ps._Z0])
coilpoint = np.array(
    [ps.perturbations_args[0]["R"], ps.perturbations_args[0]["Z"]]
)

# Simple way from opoint to xpoint then to coilpoint
Rs = np.concatenate((np.linspace(opoint[0]+1e-4, xpoint[0], n1), np.linspace(xpoint[0], coilpoint[0]-1e-4, n2)))
Zs = np.concatenate((np.linspace(opoint[1]+1e-4, xpoint[1], n1), np.linspace(xpoint[1], coilpoint[1]-1e-4, n2)))
RZs = np.array([[r, z] for r, z in zip(Rs, Zs)])

# Sophisticated way more around the xpoint
# deps = 0.05
# RZ1 = xpoint + deps * (1 - np.linspace(0, 1, n1)).reshape((n1, 1)) @ (
#     opoint - xpoint
# ).reshape((1, 2))
# RZ2 = xpoint + deps * np.linspace(0, 1, n2).reshape((n2, 1)) @ (
#     coilpoint - xpoint
# ).reshape((1, 2))
# RZs = np.concatenate((RZ1, RZ2))

# Set up the Poincare plot object
pplot = PoincarePlot(ps, pparams, integrator_params=iparams)

In [ ]:
# # R-only computation
# pplot.compute()

# R-Z computation
pplot.compute(RZs)

In [ ]:
fig_perturbed, ax_perturbed = pplot.plot(marker=".", s=1)

ax_perturbed.scatter(results[0][0], results[0][2], marker="X", edgecolors="black", linewidths=1)
ax_perturbed.scatter(ps._R0, ps._Z0, marker="o", edgecolors="black", linewidths=1)
plt.show()

In [ ]:
date = datetime.datetime.now().strftime("%m%d%H%M")
dumpname = f"../output/toybox_tok_{date}.pkl"
with open(dumpname, "wb") as f:
    pickle.dump(fig_perturbed, f)

### Back on manifold

In [6]:
iparams = dict()
iparams["rtol"] = 1e-12

manifold = Manifold(fp_perturbed, ps, integrator_params=iparams)

In [8]:
# Choose the tangles to work with
manifold.choose(0, 0)

manifold.find_N(1e-6, 1e-6)

(7, 6)

In [9]:
# Find the homoclinic points
eps_s_1, eps_u_1 = manifold.find_homoclinic(1e-8, 1e-8, n_s = 7, n_u = 7)

2024-05-06 12:06:52 [debug    ] Bounds found are : ((1.676677818914139e-09, 1e-08), (3.753209574181477e-09, 1e-08))
2024-05-06 12:06:52 [debug    ] [1.03826588e-09]
2024-05-06 12:06:52 [info     ] Root finding status : The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
2024-05-06 12:06:52 [debug    ] Root finding object :  message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 1.000e+00  1.000e+00]
       x: [-1.930e+01 -1.958e+01]
  method: hybr
    nfev: 15
    fjac: [[ 1.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00]]
       r: [-0.000e+00  0.000e+00 -0.000e+00]
     qtf: [ 1.000e+00  1.000e+00]
2024-05-06 12:06:52 [debug    ] Root finding epsilons : [4.16166109e-09 3.12339521e-09]


ValueError: Homoclinic search not successful.

In [ ]:
manifold.clinics

In [ ]:
guess_2 = [eps_s_1*np.power(manifold.lambda_s, 1/2), eps_u_1*np.power(manifold.lambda_u, 1/2)]
print(f"2nd initial guess: {guess_2}")
eps_s_2, eps_u_2 = find_homoclinic(manifold, guess_2[0], guess_2[1], n_s = 7, n_u = 7)

In [ ]:
# Plot the homoclinic points
print("\nPlotting homoclinic points")
hs_1 = manifold.integrate(manifold.rfp_s + eps_s_1 * manifold.vector_s, 7, -1)
hs_2 = manifold.integrate(manifold.rfp_s + eps_s_2 * manifold.vector_s, 7, -1)

hu_1 = manifold.integrate(manifold.rfp_u + eps_u_1 * manifold.vector_u, 7, 1)
hu_2 = manifold.integrate(manifold.rfp_u + eps_u_2 * manifold.vector_u, 7, 1)

In [ ]:
ax_perturbed.scatter(hs_1[0,:], hs_1[1,:], marker="x", color="purple", zorder=10)
ax_perturbed.scatter(hs_2[0,:], hs_2[1,:], marker="+", color="purple", zorder=10)

ax_perturbed.scatter(hu_1[0,:], hu_1[1,:], marker="x", color="blue", zorder=10)
ax_perturbed.scatter(hu_2[0,:], hu_2[1,:], marker="+", color="blue", zorder=10)

In [ ]:
print("\nComputing the manifold\n")
manifold.compute(nintersect = 9, neps = 300, epsilon=1e-7)

In [ ]:
print("\nPlotting the manifold\n")
manifold.plot(ax_perturbed, directions="u+s+")

In [ ]:
# ax_perturbed.set_xlim(6., 8)
# ax_perturbed.set_ylim(-4.6, -4)
fig_perturbed

In [ ]:
ax_perturbed.scatter(manifold.clinics[0][0], manifold.clinics[0][1], marker="o", color="magenta", zorder=10)
ax_perturbed.scatter(manifold.clinics[1][0], manifold.clinics[1][1], marker="o", color="magenta", zorder=10)

In [ ]:
fig_perturbed

In [ ]:
def resonance_area(self, n_b, n_f):
    # considering the >_u ordering of the homoclinic points
    areas = []
    for homoclinic in self.clinics:
        n_tmp_f, n_tmp_b = 1, 1

        # Forward integration
        rze_forward = homoclinic
        area_forward = []
        while n_tmp_f < n_f:
            rze_end, area_tmp = self.integrate_single(rze_forward, 1, direction=1, ret_jacobian=False, integrate_A=True)
            # if np.linalg.norm(rze_end - self.rfp_s) > np.linalg.norm(rze_forward - self.rfp_s):
            #     print("Forward integration goes away from saddle")
            rze_forward = rze_end
            ax_perturbed.scatter(rze_end[0], rze_end[1], marker="s", s=10, color="red", zorder=10)
            area_forward.append(area_tmp)
            n_tmp_f += 1
        
        # Backward integration
        rze_backward = homoclinic
        area_backward = []
        while n_tmp_b < n_b:
            rze_end, area_tmp = self.integrate_single(rze_backward, 1, direction=-1, ret_jacobian=False, integrate_A=True)
            rze_backward = rze_end
            ax_perturbed.scatter(rze_end[0], rze_end[1], marker="s", s=10, color="orange", zorder=10)
            area_backward.append(area_tmp)
            n_tmp_b += 1
        
        area_forward = np.array(area_forward)
        area_backward = np.array(area_backward)

        areas.append([area_forward, area_backward])
    return areas

In [ ]:
are = resonance_area(manifold, 8, 10)
are

In [ ]:
fig_perturbed

In [ ]:
A1 = np.concatenate((-are[0][0][:10], are[0][1][:8])).sum()

In [ ]:
A2 = np.concatenate((-are[1][0][:10], are[1][1][:8])).sum()

In [ ]:
A2-A1

In [ ]:
are[1]-are[0]

In [ ]:
areas = []
for i in range(100,300,10):
    print(f"Computing for n = {i}")
    areas.append(resonance_area(manifold, i, i))

In [ ]:
areas

In [ ]:
areas = np.array(areas)
areas

In [ ]:
plt.plot(np.arange(100,300,10), areas[:,1]-areas[:,0])

In [ ]:
# dumpname = "../output/toybox_tok_04151041.pkl"
dumpname = "../../runs/toybox-tok-shear-1504/perturbed-3-2/poincare_04161004.pkl"

# fig_perturbed = pickle.load(open("../output/toybox_.pkl", "rb"))
fig_perturbed = pickle.load(open(dumpname, "rb"))
ax_perturbed = fig_perturbed.get_axes()[0]

for col in ax_perturbed.collections:
    col.set_color('black')
    col.set_sizes([0.5])

plt.show()

In [ ]:
ax_perturbed.set_xlim(3, 9)
ax_perturbed.set_ylim(-6, 2.3)

In [ ]:
ax_perturbed.set_xlim(6., 6.4)
ax_perturbed.set_ylim(-4.6, -4.4)

In [ ]:
fig_perturbed

In [ ]:
out = manifold.stable['+'].T.flatten()
plt.plot(out[::2], out[1::2], '-o', label='stable manifold')
plt.legend()

In [ ]:
plt.plot(rzs[:,0], rzs[:,1], '-o')

In [ ]:
out = rzs_ev.T
for yy in out:
    plt.plot(yy[::2], yy[1::2], '-o')

In [ ]:
fig_perturbed, ax_perturbed = pplot.plot(marker=".", s=1)

ax_perturbed.set_xlim(6.18, 6.25)
ax_perturbed.set_ylim(-4.55, -4.45)

manifold.plot(ax_perturbed, directions="s+u+")

In [ ]:
# general view
ax_perturbed.set_xlim(2.3, 4)
ax_perturbed.set_ylim(-1.9,1)

In [ ]:
# plan americain
ax_perturbed.set_xlim(3, 3.2)
ax_perturbed.set_ylim(-1.7, -1.6)

In [ ]:
# close up view
ax_perturbed.set_xlim(3.08, 3.13)
ax_perturbed.set_ylim(-1.67, -1.63)

In [ ]:
ax_perturbed.set_xlim(4, 8)
ax_perturbed.set_ylim(-8, 3)

In [ ]:
ax_perturbed.set_xlim(6.15, 6.26)
ax_perturbed.set_ylim(-4.6, -4.4)

In [ ]:
fig_perturbed